In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from sklearn.model_selection import train_test_split
from keras.models import model_from_json
from keras import optimizers
import string
import csv
np.random.seed(1)

Using TensorFlow backend.


# Read csv file

In [0]:
data = pd.read_csv("/content/drive/My Drive/niki/train (3) (1) (3) (2).csv")

In [4]:

print(data.shape)

(5693, 6)


In [5]:
#reason for not using Review title is because 5000 of them are none
sub_data = data[['Review Text','Star Rating']] 
print(sub_data.shape)

(5693, 2)


In [6]:
sub_data.replace('', np.nan, inplace=True)
print(sub_data.shape)

(5693, 2)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)


In [7]:
sub_data.dropna( inplace=True)
print(sub_data.shape)

(5692, 2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Remove Duplicates

In [8]:
sub_data.duplicated(subset={'Review Text','Star Rating'},).value_counts()
sub_data.drop_duplicates(subset={'Review Text','Star Rating'},keep="first",inplace=True)
print(sub_data.shape)

(4659, 2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
print(sub_data.head())

                                         Review Text  Star Rating
0                   Very bad wallet balance not use.            1
1  Froud app i recharge 199 but not done also sen...            1
2                       Waste to write comments also            2
3                                           Nice apo            5
4                                      Good nice app            5


# Split into training and validation data

In [10]:

X_train, X_test, y_train, y_test = train_test_split(np.array(sub_data[['Review Text']]),
                                                    np.array(sub_data[['Star Rating']]), test_size=0.1, 
                                                    random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(4193, 1) (4193, 1)
(466, 1) (466, 1)


In [0]:
def convert_to_one_hot(Y, C):
    Y_hot = np.zeros((Y.shape[0],C))
    for i in range(Y.shape[0]):
      Y_hot[i,Y[i][0]-1] = 1
    
    return Y_hot

In [12]:
y_train = convert_to_one_hot(y_train,5)
y_test = convert_to_one_hot(y_test,5)
print(y_train.shape, y_test.shape)

(4193, 5) (466, 5)


In [0]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

# Read glove word2vec

In [0]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('/content/drive/My Drive/toxicity/glove.6B.200d.txt')

In [15]:
word = "0"
print(word_to_index[word])

1681


In [16]:
import string
max_len = 0
avg_len = 0
num = 0
for i in np.array(sub_data[['Review Text']]):
  i = i[0]
  #print(i)
  tmp = len(i.translate(str.maketrans('','',string.punctuation)).lower().split())
  num+=1
  avg_len += tmp
  if tmp>max_len:
    max_len = tmp
  #print(tmp)
avg_len = avg_len/num
print(max_len)
print(avg_len)
print(num)

365
15.23846318952565
4659


In [0]:

def sentences_to_indices(X, word_to_index, max_len):
    m = X.shape[0]                                
    X_indices = np.zeros((m, max_len))
    
    for i in range(m): 
        #print(X[i])
        tmp = X[i][0].translate(str.maketrans('','',string.punctuation)).lower().split()
        j = 0
        for w in tmp:
          if j>= max_len:
            break
          try:
            X_indices[i, j] = word_to_index[w]
            j = j+1
            #print(w)
          except KeyError:
            #print(w)
            pass
            
    
    return X_indices

In [18]:
X_train[0]

array(["Useless App..does not support a lot of payments that other apps are supporting already. Don't waste your time like i did. Does not even support bsnl bill payment"],
      dtype=object)

In [0]:
avg_len = int(avg_len)+1

In [20]:
X_train_indices = sentences_to_indices(X_train,word_to_index, avg_len)
X_test_indices = sentences_to_indices(X_test,word_to_index, avg_len)

print(X_train_indices.shape)
print(X_test_indices.shape)

(4193, 16)
(466, 16)


In [0]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1            
    emb_dim = word_to_vec_map["cucumber"].shape[0]  
    emb_matrix = np.zeros((vocab_len, emb_dim))
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
    
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
#    embedding_layer = Embedding(vocab_len,
#                             emb_dim,
#                             weights=[emb_matrix],
#                             trainable=False)
    
    return embedding_layer

# Intialise embedding layer with Glove word2vec

In [22]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print(embedding_layer)

# Build the network

In [0]:
def network(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices) 
    X = LSTM(256, return_sequences=True)(embeddings)
    #X = Dropout(0.6)(X)
    X = LSTM(256, return_sequences=False)(X)
    #X = Dropout(0.6)(X)
    X = Dense(5)(X)
    X =  Activation('softmax')(X)
    model = Model(inputs=sentence_indices, outputs=X)
    
    return model

In [35]:
model = network((None,), word_to_vec_map, word_to_index)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, None, 200)         80000200  
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 256)         467968    
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 80,994,765
Trainable params: 994,565
Non-trainable params: 80,000,200
_________________________________________

In [0]:
sgd = optimizers.SGD(lr=0.1, decay=1e-4, momentum=0.9, nesterov=True)
adam =optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999,  decay=0)
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

# Fit the network

In [0]:
#class_weight ={0:2083/1729,1:2083/153,2:2083/204,3:2083/490,4:1}

model.fit(X_train_indices, y_train,validation_data=(X_test_indices, y_test), epochs = 45,
          batch_size = 64, shuffle=True)#,class_weight = class_weight)

# Weighted F-Score

In [38]:
from sklearn.metrics import f1_score
y_true = np.argmax(y_test, axis=1)
y_pred = np.argmax(model.predict(X_test_indices), axis=1)
print(f1_score(y_true, y_pred, average='weighted')  )

0.6517983778019384


In [41]:
model_json = model.to_json()
with open("/content/drive/My Drive/niki/model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("/content/drive/My Drive/niki/model.h5")
print("Saved model to disk")

Saved model to disk


In [42]:

json_file = open("/content/drive/My Drive/niki/model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("/content/drive/My Drive/niki/model.h5")
print("Loaded model from disk")

loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Loaded model from disk


In [43]:
loaded_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, None, 200)         80000200  
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 256)         467968    
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 1285      
_________________________________________________________________
activation_2 (Activation)    (None, 5)                 0         
Total params: 80,994,765
Trainable params: 994,565
Non-trainable params: 80,000,200
_________________________________________

In [0]:
def text_sentence_to_indices(text_sentence, word_to_index):
    tmp = text_sentence.translate(str.maketrans('','',string.punctuation)).lower().split()
    j = 0
    X_indices = np.zeros((1, len(tmp)))
    for w in tmp:
      try:
        X_indices[0, j] = word_to_index[w]
        j = j+1
        #print(w)
      except KeyError:
        #print(w)
        pass
            
    
    return X_indices

In [39]:

s = "Awesome app, all should use it"
ind = text_sentence_to_indices(s,word_to_index)
print(ind)

[[ 64354.  57667.  51582. 329916. 374163. 193716.]]


In [44]:
loaded_model.predict(ind)

array([[1.4275568e-02, 5.2325381e-04, 3.4637565e-03, 1.8413547e-02,
        9.6332389e-01]], dtype=float32)

# Read test data

In [45]:
test_data = pd.read_csv("/content/drive/My Drive/niki/test (3) (1) (3) (2).csv")
sub_test_data = test_data[['id','Review Text']] 
print(sub_test_data.shape)
sub_test_data.replace('', np.nan, inplace=True)
print(sub_test_data.shape)
sub_test_data.dropna( inplace=True)
print(sub_test_data.shape)

(1424, 2)
(1424, 2)
(1423, 2)


/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Predict on test data

In [0]:
csv_data = [["id","Star Rating"]]
sub_test_data = np.array(sub_test_data)
for i in range(sub_test_data.shape[0]):
  ind = text_sentence_to_indices(sub_test_data[i,1],word_to_index)
  pred_label = np.argmax(loaded_model.predict(ind))+1
  csv_data.append([sub_test_data[i,0],pred_label])

# Save the predictions

In [0]:
with open("/content/drive/My Drive/niki/predictions.csv", 'w') as csvfile: 
    csvwriter = csv.writer(csvfile) 
    csvwriter.writerows(csv_data)

# References:


* Deep Learning by Andrew Ng, coursera 

